In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from transformers import AutoModel
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'ScientificName']]
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model(image_tensor).last_hidden_state.mean(dim=1)

    
    return features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...") ])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ...: 100%|██████████| 11399/11399 [36:30<00:00,  5.20it/s]  


Extracted feature shape: (11399, 768)


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128), activation='logistic', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():

    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 61.75% | Prec: 70.72% | Rec: 61.75% | F1: 63.87% | Bal Acc: 54.78% | MCC: 0.5999
LogisticRegression        | Acc: 94.12% | Prec: 93.91% | Rec: 94.12% | F1: 93.71% | Bal Acc: 76.17% | MCC: 0.9372
SVMLinear                 | Acc: 94.30% | Prec: 94.23% | Rec: 94.30% | F1: 93.87% | Bal Acc: 77.48% | MCC: 0.9391
SVMPolynomial             | Acc: 62.37% | Prec: 73.36% | Rec: 62.37% | F1: 59.27% | Bal Acc: 31.21% | MCC: 0.6122
SVMRadialBasis            | Acc: 90.39% | Prec: 87.38% | Rec: 90.39% | F1: 88.16% | Bal Acc: 54.57% | MCC: 0.8973
NearestNeighbor           | Acc: 78.51% | Prec: 76.65% | Rec: 78.51% | F1: 76.42% | Bal Acc: 47.36% | MCC: 0.7693
RandomForest              | Acc: 68.82% | Prec: 65.79% | Rec: 68.82% | F1: 62.80% | Bal Acc: 23.88% | MCC: 0.6649
MLP_Baseline              | Acc: 94.96% | Prec: 94.53% | Rec: 94.96% | F1: 94.46% | Bal Acc: 78.61% | MCC: 0.9461


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus impunctatus,Cymindis cribricollis,Synuchus impunctatus,Synuchus impunctatus,Calathus advena,Synuchus impunctatus,Synuchus impunctatus,Calathus advena,Synuchus impunctatus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum punctiforme,Agonum punctiforme,Calathus advena,Calathus advena,Calathus advena,Synuchus impunctatus,Synuchus impunctatus,Calathus advena,Agonum punctiforme


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/19.DINOv2-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/19.DINOv2-linear-probing-species-metrics.csv", index=False)